In [1]:
""" Path to project /Users/a.ovsiannikova/PycharmProjects/HIV_immunogenicity"""

' Path to project /Users/a.ovsiannikova/PycharmProjects/HIV_immunogenicity'

In [2]:
%matplotlib inline
from util.aa_properties import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from pandas.plotting import scatter_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [3]:
# "TCR contact residue hydrophobicity is a hallmark of immunogenic CD8+ T cell epitopes" dataset
# TODO: link here

dataset = pd.read_csv('data/pnas_dataset.csv', sep=';')

In [4]:
# some of the columns are read with leading space - replacing column names

dataset.columns = ['Epitope', 'Epitope Start', 'Epitope End', 'MHC Allele',
       'Epitope Source Organism Name', 'Length', 'Immunogenicity']

In [5]:
# function for mice filtering from the dataset

def filter_mice(seq):
    return 'Human' if seq.startswith('HLA') else 'Mouse'

In [6]:
# creating Host column - 'Human' or 'Mouse'

dataset['Host'] = dataset['MHC Allele'].apply(filter_mice)

In [7]:
# keeping only Human hosts

dataset = dataset.loc[dataset.Host == 'Human']

In [8]:
# adding amino acid score (sum)
dataset['aa_score'] = 0
dataset['aa_score'] = dataset.Epitope.apply(aaprop_sequence)

# adding kidera score (sum)
dataset['kidera_score'] = 0
dataset['kidera_score'] = dataset.Epitope.apply(score_sequence)

# adding hydrophobicity score (sum)
dataset['hydrophobicity_score'] = 0
dataset['hydrophobicity_score'] = dataset.Epitope.apply(score_hydrophobicity_sequence)

In [9]:
# function to score epitopes by all available amino acid parameters

def score_acids(ds):
    # for each column in amino acid data 
    for column in aa_properties.columns:
        
        # adding sum of column
        ds[column] = ds.apply(lambda x: aa_properties[column].loc[list(x.Epitope)].sum(), axis=1)
    return ds

In [10]:
# separate amino acid score

dataset = score_acids(dataset)

In [11]:
dataset.head(25)

,Epitope,Epitope Start,Epitope End,MHC Allele,Epitope Source Organism Name,Length,Immunogenicity,Host,aa_score,kidera_score,...,kf1,kf2,kf3,kf4,kf5,kf6,kf7,kf8,kf9,kf10
0,KLEDLERDL,26,34,HLA-A*02:01,Hepatitis delta virus TW2667,9,Positive,Human,11.05,-4.98,...,-4.98,2.03,-5.96,4.23,-6.27,-6.21,1.99,-1.050000e+00,1.92,5.48
1,DLMGYIPLV,132,140,HLA-A*02:01,Hepatitis C virus subtype 1a,9,Positive,Human,8.79,0.53,...,0.53,-1.72,0.77,-4.76,0.92,-4.45,0.07,7.200000e-01,-0.38,2.41
2,QTVTSTPVQGR,792,802,HLA-A*68:01,Human herpesvirus 5,11,Positive,Human,9.68,2.91,...,2.91,-5.14,8.00,4.67,1.97,-0.34,0.91,-6.230000e+00,-4.32,-1.91
3,TTVYPPSSTAK,945,955,HLA-A*03:01,Human herpesvirus 5,11,Positive,Human,9.29,5.26,...,5.26,-5.00,3.29,1.70,2.15,-5.02,-2.89,-9.420000e+00,-2.92,0.85
4,LITGRLQSL,978,986,HLA-A2,SARS coronavirus,9,Positive,Human,9.30,-1.57,...,-1.57,-2.39,3.65,-0.21,-3.00,-5.40,3.48,-2.810000e+00,-0.98,0.03
7,AVAKAGKPL,239,247,HLA-E*01:01,Salmonella typhi Ty21a,9,Positive,Human,9.62,-3.62,...,-3.62,-6.37,-2.95,0.18,1.22,-9.00,3.21,-1.040000e+00,-1.26,1.52
8,AMLQDIATL,287,295,HLA-E*01:01,Salmonella typhi Ty21a,9,Positive,Human,10.75,-6.96,...,-6.96,-4.00,-1.35,-1.70,-1.42,-3.16,0.51,-2.450000e+00,0.88,-2.05
9,KMLRGVNVL,15,23,HLA-E*01:01,Salmonella typhi Ty21a,9,Positive,Human,9.54,-2.48,...,-2.48,-1.18,3.97,0.49,2.02,-5.10,3.34,1.830000e+00,-2.10,3.69
10,KLQERVAKL,364,372,HLA-E*01:01,Salmonella typhi Ty21a,9,Positive,Human,10.93,-6.76,...,-6.76,0.96,-0.04,4.67,-0.36,-7.48,4.75,-2.420000e+00,-0.46,1.71
11,ITKKVADLVGF,102,112,HLA-B*57:02,Homo sapiens,11,Positive,Human,11.33,-3.40,...,-3.40,-3.51,3.24,0.31,1.36,-8.22,0.80,2.410000e+00,-0.31,2.08


In [12]:
# check if there are duplicate epitopes
# TODO: there are, all kept - will change soon

nonunique = dataset.groupby('Epitope').size().loc[dataset.groupby('Epitope').size() >= 2].index

In [13]:
# duplicate epitopes data

dataset.loc[dataset.Epitope.isin(nonunique)].sort_values('Epitope')

,Epitope,Epitope Start,Epitope End,MHC Allele,Epitope Source Organism Name,Length,Immunogenicity,Host,aa_score,kidera_score,...,kf1,kf2,kf3,kf4,kf5,kf6,kf7,kf8,kf9,kf10
112,ALSDHHIYL,216,224,HLA-A*02:01,Homo sapiens,9,Positive,Human,9.88,-2.42,...,-2.42,-0.61,-0.84,-2.01,-0.48,-3.79,-6.53,2.000000e-01,3.77,3.86
5733,ALSDHHIYL,216,224,HLA-A2,Homo sapiens,9,Negative,Human,9.88,-2.42,...,-2.42,-0.61,-0.84,-2.01,-0.48,-3.79,-6.53,2.000000e-01,3.77,3.86
20,FLFDGSPTYVL,2335,2345,HLA-A*02:01,Homo sapiens,11,Positive,Human,10.13,3.31,...,3.31,-1.56,0.05,-5.07,-0.41,-7.84,0.33,4.440892e-16,2.12,3.00
5705,FLFDGSPTYVL,2335,2345,HLA-A2,Homo sapiens,11,Negative,Human,10.13,3.31,...,3.31,-1.56,0.05,-5.07,-0.41,-7.84,0.33,4.440892e-16,2.12,3.00
2594,GIVEQCCTSI,90,99,HLA-A*02:01,Homo sapiens,10,Positive,Human,9.98,-1.35,...,-1.35,-6.12,6.12,-0.88,-2.42,4.73,0.86,-5.700000e-01,-0.45,-2.74
5990,GIVEQCCTSI,90,99,HLA-A*02:01,Homo sapiens,10,Negative,Human,9.98,-1.35,...,-1.35,-6.12,6.12,-0.88,-2.42,4.73,0.86,-5.700000e-01,-0.45,-2.74
107,GLLGTLVQL,400,408,HLA-A*02:01,Homo sapiens,9,Positive,Human,9.32,-2.19,...,-2.19,-5.09,1.90,-3.39,-0.50,-8.43,6.49,-1.200000e-01,-2.57,3.15
5321,GLLGTLVQL,400,408,HLA-A2,Homo sapiens,9,Negative,Human,9.32,-2.19,...,-2.19,-5.09,1.90,-3.39,-0.50,-8.43,6.49,-1.200000e-01,-2.57,3.15
110,SLFPGKLEV,1010,1018,HLA-A*02:01,Homo sapiens,9,Positive,Human,9.15,-0.49,...,-0.49,-2.09,-1.86,-1.65,0.62,-7.93,2.44,-1.150000e+00,-0.57,2.50
6610,SLFPGKLEV,1010,1018,HLA-A2,Homo sapiens,9,Negative,Human,9.15,-0.49,...,-0.49,-2.09,-1.86,-1.65,0.62,-7.93,2.44,-1.150000e+00,-0.57,2.50


In [14]:
# is this ok?

dataset.loc[dataset['Epitope Source Organism Name'] == 'Mus musculus']

,Epitope,Epitope Start,Epitope End,MHC Allele,Epitope Source Organism Name,Length,Immunogenicity,Host,aa_score,kidera_score,...,kf1,kf2,kf3,kf4,kf5,kf6,kf7,kf8,kf9,kf10
1570,LLGRDSFEV,261,269,HLA-A*02:01,Mus musculus,9,Positive,Human,9.40,-1.41,...,-1.41,-1.53,-0.69,0.08,-4.42,-5.25,0.39,1.31,0.16,3.81
2788,RLFGIDLLWSV,226,236,HLA-A*02:01,Mus musculus,11,Positive,Human,11.22,-1.43,...,-1.43,0.22,1.75,-4.53,-5.36,-7.10,0.00,0.28,-0.68,2.48
2789,LFGIDLLWSV,227,236,HLA-A*02:01,Mus musculus,10,Positive,Human,10.26,-1.65,...,-1.65,-1.05,0.38,-6.40,-3.66,-7.56,-0.92,0.67,-0.91,1.55
2790,FGIDLLWSV,228,236,HLA-A*02:01,Mus musculus,9,Positive,Human,8.96,-0.61,...,-0.61,-1.05,0.62,-5.30,-3.11,-5.51,-1.88,1.43,-1.36,0.62
3082,LLVHFLPLL,3,11,HLA-A*02:01,Mus musculus,9,Positive,Human,10.22,-4.50,...,-4.50,0.46,-0.95,-7.80,0.46,-11.31,2.85,-4.47,5.37,6.21
3083,HLCGPHLVEA,29,38,HLA-A*02:01,Mus musculus,10,Positive,Human,10.87,-3.01,...,-3.01,-4.33,-2.51,-3.10,0.65,-1.42,0.13,-0.85,2.77,6.45
3084,IVDQCCTSI,89,97,HLA-A*02:01,Mus musculus,9,Positive,Human,9.02,-0.78,...,-0.78,-4.57,6.38,-1.08,-2.13,4.59,-2.02,-2.84,2.32,-2.38
3086,ALWEPKPTQ,15,23,HLA-A*02:01,Mus musculus,9,Positive,Human,9.38,-0.18,...,-0.18,0.32,-4.79,0.16,0.35,-3.58,3.15,-7.40,-1.58,-2.37
4791,AINSEMFLR,272,280,HLA-A*11:01,Mus musculus,9,Positive,Human,10.22,-4.22,...,-4.22,-0.36,-0.40,-0.03,-2.84,-1.29,-0.16,0.95,1.77,-2.65
4792,TLALEVAQQK,79,88,HLA-A*11:01,Mus musculus,10,Positive,Human,12.12,-8.41,...,-8.41,-3.26,-0.87,2.56,-0.13,-6.30,3.56,-3.89,-0.59,-2.44


In [15]:
# self immunogenicity?

dataset.loc[(dataset['Epitope Source Organism Name'] == 'Homo sapiens') & (dataset.Immunogenicity == 'Positive')]

,Epitope,Epitope Start,Epitope End,MHC Allele,Epitope Source Organism Name,Length,Immunogenicity,Host,aa_score,kidera_score,...,kf1,kf2,kf3,kf4,kf5,kf6,kf7,kf8,kf9,kf10
11,ITKKVADLVGF,102,112,HLA-B*57:02,Homo sapiens,11,Positive,Human,11.33,-3.40,...,-3.40,-3.51,3.24,0.31,1.36,-8.22,0.80,2.410000e+00,-0.31,2.08
13,SRHHAFCFR,666,674,HLA-B27,Homo sapiens,9,Positive,Human,9.72,-1.43,...,-1.43,1.90,1.10,0.54,-1.59,2.52,-1.09,4.400000e-01,6.51,4.63
15,LIYDSSLCDL,83,92,HLA-A2,Homo sapiens,10,Positive,Human,10.42,0.43,...,0.43,-2.17,-0.52,-3.22,-5.16,-4.95,-3.56,-2.790000e+00,2.67,3.58
19,GLIEKNIEL,425,433,HLA-A2,Homo sapiens,9,Positive,Human,10.11,-4.18,...,-4.18,-1.15,-0.70,0.95,-2.98,-4.60,2.72,3.770000e+00,0.48,-2.61
20,FLFDGSPTYVL,2335,2345,HLA-A*02:01,Homo sapiens,11,Positive,Human,10.13,3.31,...,3.31,-1.56,0.05,-5.07,-0.41,-7.84,0.33,4.440892e-16,2.12,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4911,LLIDLTSFL,107,115,HLA-A*02:01,Homo sapiens,9,Positive,Human,9.92,-3.45,...,-3.45,-1.18,0.26,-4.74,-3.75,-9.05,0.51,-3.260000e+00,3.40,2.16
4912,LLSLFSLWL,115,123,HLA-A*02:01,Homo sapiens,9,Positive,Human,10.20,-3.49,...,-3.49,0.92,-1.96,-7.66,-4.11,-11.35,1.21,-5.400000e+00,-0.28,3.15
4913,LLSILCIWV,145,153,HLA-A*02:01,Homo sapiens,9,Positive,Human,9.67,-4.09,...,-4.09,-0.90,4.79,-7.44,-4.31,-4.35,-0.70,-3.440000e+00,0.07,0.15
4918,QLLNSVLTL,248,256,HLA-A*02:01,Homo sapiens,9,Positive,Human,9.92,-3.16,...,-3.16,-2.32,2.40,-1.84,-0.73,-8.27,1.87,-4.760000e+00,0.88,0.27


In [16]:
# collecting unique alleles 
# TODO: need to use unique()

alleles = dataset.groupby('MHC Allele').size().index

In [17]:
# keeping only 6 first symbols from MHC Allelle (should be: HLA-S*XX:XX)
# TODO: 'HLA-A23' 'HLA-Cw6' 'HLA-E*01033 HLA-Class I, allele undetermined' 'HLA-B*44:27 (HLA-B*4427)' 'HLA-A3/11' etc

alleles_tmp = {allele: allele[:6] for allele in alleles}

In [18]:
set(alleles_tmp.values())

{'HLA-A*',
 'HLA-A1',
 'HLA-A2',
 'HLA-A3',
 'HLA-A6',
 'HLA-B*',
 'HLA-B1',
 'HLA-B2',
 'HLA-B3',
 'HLA-B4',
 'HLA-B5',
 'HLA-B6',
 'HLA-B7',
 'HLA-B8',
 'HLA-C*',
 'HLA-Cl',
 'HLA-Cw',
 'HLA-E',
 'HLA-E*'}

In [20]:
# encoder for alleles to get ints

le = LabelEncoder()
le.fit(list(set(alleles_tmp.values())))

# first, mapping alleles to their short versions
dataset['allele'] = dataset['MHC Allele'].map(alleles_tmp)

# encoding short names
dataset['Allele'] = le.transform(dataset['allele'])

In [21]:
# encoder for organisms
# TODO: group organisms?

le_organisms = LabelEncoder()
le_organisms.fit(dataset['Epitope Source Organism Name'])
dataset['Organism'] = le_organisms.transform(dataset['Epitope Source Organism Name'])

In [22]:
# target encoding 

encode_target = {'Positive': 1, 'Negative': 0}
dataset['target'] = dataset.Immunogenicity.map(encode_target)

In [23]:
# seems like organism column is split too much - a lot of unique organisms

dataset.groupby(['Epitope Source Organism Name']).size().to_frame().sort_values('Epitope Source Organism Name').head(30)

,0
Epitope Source Organism Name,
Adenoviridae,1
Alcaligenes sp.,1
Andes virus,3
Aspergillus fumigatus,3
BK polyomavirus,37
Brucella abortus,1
Chlamydia pneumoniae,3
Chlamydia trachomatis,18
Chlamydophila pneumoniae CWL029,23


In [24]:
# keeping only numeric columns

df_train = dataset[['Epitope Start', 'Organism', 'Length', 'aa_score', 'kidera_score', 'hydrophobicity_score', 
                    'alpha', 'beta', 'charge', 'core', 'hydropathy', 'pH', 'polarity', 'rim', 'surface',
                    'turn', 'volume', 'count', 'strength', 'disorder', 'mjenergy', 'kf1',
                    'kf2', 'kf3', 'kf4', 'kf5', 'kf6', 'kf7', 'kf8', 'kf9', 'kf10',
                    'target', 'Allele']]

In [25]:
# making train - test split with stratification by target value and shuffling
# TODO: set random state?

X_train, X_test, y_train, y_test = train_test_split(df_train.drop('target', axis=1), df_train.target, test_size=0.3, 
                                                    stratify=df_train.target, shuffle=True)

In [26]:
# scaling features. scaler is fitted on train data to prevent data leak

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

In [27]:
# random forest to try
# TODO: boolean true?
# no need in test set actually as oob scoring can be performed

forest = RandomForestClassifier(n_estimators = 350, max_depth = 15, max_features = 'sqrt', 
                                bootstrap='True', n_jobs = -1, oob_score=True, min_samples_leaf=5, 
                                min_samples_split=5, random_state=6)

In [28]:
# fitting with train data

forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap='True', class_weight=None, criterion='gini',
                       max_depth=15, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=350,
                       n_jobs=-1, oob_score=True, random_state=6, verbose=0,
                       warm_start=False)

In [29]:
# too lazy to check on test
# TODO: one day...

forest.oob_score_

0.9479932165065008

In [30]:
# feature importance can indicate if there is any kind of overfitting
# seems there is: feature[1] and feature[-1] have large weights

forest.feature_importances_

array([0.03989642, 0.42854519, 0.0035122 , 0.00771969, 0.00798827,
       0.0280763 , 0.00846064, 0.01023462, 0.00449255, 0.00872937,
       0.03514005, 0.00680682, 0.02733456, 0.02170409, 0.01254793,
       0.00944392, 0.00797167, 0.00344516, 0.02449113, 0.00802568,
       0.01900346, 0.00842427, 0.00834821, 0.00860247, 0.06705372,
       0.009455  , 0.01356821, 0.01577891, 0.01071483, 0.00917972,
       0.01178017, 0.11352477])

In [31]:
# mapping feature names to scores

feature_importance = {feature: importance for (feature, importance) in zip(df_train.drop('target', axis=1).columns, forest.feature_importances_)}

In [32]:
# sorting in descending order

feature_importance = sorted(feature_importance, key=lambda x: feature_importance[x], reverse=True)

In [33]:
feature_importance

['Organism',
 'Allele',
 'kf4',
 'Epitope Start',
 'hydropathy',
 'hydrophobicity_score',
 'polarity',
 'strength',
 'rim',
 'mjenergy',
 'kf7',
 'kf6',
 'surface',
 'kf10',
 'kf8',
 'beta',
 'kf5',
 'turn',
 'kf9',
 'core',
 'kf3',
 'alpha',
 'kf1',
 'kf2',
 'disorder',
 'kidera_score',
 'volume',
 'aa_score',
 'pH',
 'charge',
 'Length',
 'count']

In [34]:
# lets try to remove some of the most important features

forest_wo_important = RandomForestClassifier(n_estimators = 350, max_depth = 15, max_features = 'sqrt', 
                                             bootstrap = 'True', n_jobs = -1, oob_score=True, min_samples_leaf=5, 
                                             min_samples_split=5, random_state=6)

In [35]:
# top 5 features are removed

X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(df_train.drop(feature_importance[:5] + ['target'], axis=1), df_train.target, test_size=0.3, 
                                                    stratify=df_train.target, shuffle=True)

In [36]:
forest_wo_important.fit(X_train_i, y_train_i)

RandomForestClassifier(bootstrap='True', class_weight=None, criterion='gini',
                       max_depth=15, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=350,
                       n_jobs=-1, oob_score=True, random_state=6, verbose=0,
                       warm_start=False)

In [37]:
# still quite good

forest_wo_important.oob_score_

0.7456189937817976

In [38]:
# no overfitting detected

forest_wo_important.feature_importances_

array([0.01202506, 0.02683709, 0.02877052, 0.08410198, 0.02794068,
       0.03901103, 0.01113782, 0.02890833, 0.02049243, 0.05874414,
       0.07072346, 0.04465372, 0.03382333, 0.02929276, 0.01114926,
       0.06772776, 0.02525673, 0.07016687, 0.02972304, 0.03004028,
       0.03124549, 0.03111086, 0.04020222, 0.04117063, 0.03855207,
       0.03002537, 0.03716706])

In [39]:
# just checking correlations

df_train.corr().loc['target'].sort_values()

kf4                    -0.444057
polarity               -0.407254
rim                    -0.344731
mjenergy               -0.320355
disorder               -0.298251
surface                -0.284620
turn                   -0.181708
aa_score               -0.128705
alpha                  -0.128705
Length                 -0.122908
count                  -0.122908
kf2                    -0.106454
Allele                 -0.105212
kf6                    -0.098253
volume                 -0.095211
kf8                    -0.092800
pH                     -0.040037
core                   -0.030844
kf5                    -0.024727
kidera_score           -0.002094
kf1                    -0.002094
charge                  0.021734
beta                    0.032279
kf3                     0.093755
kf9                     0.104520
kf7                     0.135719
kf10                    0.137432
Epitope Start           0.146241
Organism                0.289148
strength                0.378742
hydrophobi

In [300]:
# need to choose features to use 

# dendrogram = sch.dendrogram(sch.linkage(X_train, method='ward'))

In [ ]:
# distributions
# see data/scatter.jpg

sm = scatter_matrix(df_train, alpha = 0.2, figsize = (50, 50), diagonal = 'kde')

In [ ]:
# distributions by target
# see data/target coloured.png

shape = len(df_train.columns)
k = 1
fig = plt.figure(figsize=(120, 120))
for i, feature_1 in enumerate(df_train.columns):
    for j, feature_2 in enumerate(df_train.columns):
        plt.subplot(shape, shape, k)
        plt.scatter(df_train.loc[df_train.target == 1][feature_1], df_train.loc[df_train.target == 1][feature_2], s=10, marker='o', color='red', alpha=0.05)
        plt.scatter(df_train.loc[df_train.target == 0][feature_1], df_train.loc[df_train.target == 0][feature_2], s=10, marker='o', color='blue', alpha=0.05)
#         plt.scatter(normed_res[labels==2][feature_1], normed_res[labels==2][feature_2], s=50, marker='o', color='green')
        plt.title(f'{feature_1} vs {feature_2}')
        plt.xlabel(feature_1)
        plt.ylabel(feature_2)
        k += 1
plt.tight_layout()
plt.savefig('data/target coloured.png', quality=100)

In [283]:
# model = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
# model.fit(df_train)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='ward', memory=None, n_clusters=3,
                        pooling_func='deprecated')